In [71]:
import os, sys, time
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold

sys.path.append('/home/ymm/kaggle/xgboost_hyperopt')
import utils.bosch_functions as bosch_functions
from utils.wrapped_xgboost import xgboost_classifier
from utils.validation_tools import score_MCC, MCC, create_validation_index
from utils.models import CombinedModel
from utils.data_munge import remove_single_value_columns
from utils.feature_engineering import NumericalFeatureEngineering, getRelativeTimeColumns, BasicDate_FeatureEngineering
from utils.feature_engineering import getTimeChangeColumns, getTimeSteps, build_IndexFeatures

data_path = '/home/ymm/bosch/'

train_num_file   = 'train_numeric.csv'
train_cat_file   = 'train_categorical.csv'
train_date_file  = 'train_date.csv'
test_num_file    = 'test_numeric.csv'
test_cat_file    = 'test_categorical.csv'
test_date_file   = 'test_date.csv'

sample_submission_file   = 'sample_submission.csv'

start_time_column_name = 'L0_S0_D1'
id_column_name = 'Id'
dep_var_name = 'Response'

In [2]:
#'''
num_rows = 150000
start_time = time.time()
## loading the data by using the skipped_row_num list
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id', nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id', nrows=num_rows)
train_cat = pd.read_csv(join(data_path, train_cat_file),    index_col='Id', nrows=num_rows)
test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id', nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id', nrows=num_rows)
test_cat = pd.read_csv(join(data_path, test_cat_file),      index_col='Id', nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
#'''

finish loading date using 164.0 seconds


In [3]:
'''
bin_num = 1 ## number of bins to separate data by start_time
tmp_train, tmp_test, bins, bin_names = bosch_functions.create_grouped_index_df(bin_num)
#none_selected_window_num = ['0']
none_selected_window_num = [np.NaN]
skipped_test_row_num = tmp_test.loc[tmp_test['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()
skipped_train_row_num = tmp_train.loc[tmp_train['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()


num_rows = 50000
start_time = time.time()
## loading the data by using the skipped_row_num list
train_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)
train_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)
test_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id',  skiprows=skipped_test_row_num,  nrows=num_rows)
test_dat = pd.read_csv(join(data_path, test_date_file),     index_col='Id',  skiprows=skipped_test_row_num,  nrows=num_rows)

print 'finish loading date using {} seconds'.format(round(time.time() - start_time, 0))
'''

"\nbin_num = 1 ## number of bins to separate data by start_time\ntmp_train, tmp_test, bins, bin_names = bosch_functions.create_grouped_index_df(bin_num)\n#none_selected_window_num = ['0']\nnone_selected_window_num = [np.NaN]\nskipped_test_row_num = tmp_test.loc[tmp_test['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()\nskipped_train_row_num = tmp_train.loc[tmp_train['time_window_num'].isin(none_selected_window_num), 'row_num'].tolist()\n\n\nnum_rows = 50000\nstart_time = time.time()\n## loading the data by using the skipped_row_num list\ntrain_num = pd.read_csv(join(data_path, train_num_file),    index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)\ntrain_dat = pd.read_csv(join(data_path, train_date_file),   index_col='Id',  skiprows=skipped_train_row_num, nrows=num_rows)\ntest_num = pd.read_csv(join(data_path, test_num_file),      index_col='Id',  skiprows=skipped_test_row_num,  nrows=num_rows)\ntest_dat = pd.read_csv(join(data_path, test_date_file), 

In [4]:
remove_single_value_columns(train_num, 'Response', test=test_num)
remove_single_value_columns(train_dat, test=test_dat)
remove_single_value_columns(train_cat, test=test_cat)

raw train data dimension:  (150000, 969)
raw test data dimension:  (150000, 968)
processed train data dimension:  (150000, 969)
processed test data dimension:  (150000, 968)
raw train data dimension:  (150000, 1156)
raw test data dimension:  (150000, 1156)
processed train data dimension:  (150000, 1150)
processed test data dimension:  (150000, 1150)
raw train data dimension:  (150000, 2140)
raw test data dimension:  (150000, 2140)
processed train data dimension:  (150000, 1294)
processed test data dimension:  (150000, 1294)


In [5]:
dat_columns = train_dat.columns.tolist()
num_columns = train_num.columns.tolist()
num_columns.remove(dep_var_name)

def build_column_dict(columns):
    col_dict = {}
    for col in columns:
        tmpList = col.split('_')[0:2]
        key = ('_').join(tmpList)
        if key not in col_dict:
            col_dict[key] = [col]
        else:
            col_dict[key].append(col)
            
    return col_dict


def build_station_features(df, col_dict, prefix='dat'):
    features = pd.DataFrame()
    for key, value in col_dict.items():
        features['{}_{}_{}'.format(prefix, key, 'mean')] = df[value].mean(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'max')] = df[value].max(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'min')] = df[value].min(axis=1)
        features['{}_{}_{}'.format(prefix, key, 'var')] = df[value].var(axis=1)
    return features
    
    
dat_col_dict = build_column_dict(dat_columns)
num_col_dict = build_column_dict(num_columns)
        

In [6]:
start_time = time.time()

train_dat_stations = build_station_features(train_dat, dat_col_dict, 'dat')
test_dat_stations = build_station_features(test_dat, dat_col_dict, 'dat')

train_num_stations = build_station_features(train_num, num_col_dict, 'num')
test_num_stations = build_station_features(test_num, num_col_dict, 'num')

print 'finish feature engineering date station using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date station using 0.89 minutes


#### build categorical features

In [7]:
def BasicCat_FeatureEngineering(train_cat):
    ## feature engineering on the date features
    encoder = preprocessing.LabelEncoder()
    column_names = train_cat.columns.tolist()
    column_names.append('NaN')
    encoder.fit(column_names)
    dat_new_fea = pd.DataFrame()
    dat_new_fea['cat_sum'] = train_cat.sum(axis=1)
    dat_new_fea['cat_mean'] = train_cat.mean(axis=1)
    dat_new_fea['cat_nan_count'] = train_cat.isnull().sum(axis=1)
    dat_new_fea['cat_max'] = train_cat.max(axis=1)
    dat_new_fea['cat_min'] = train_cat.min(axis=1)
    dat_new_fea['cat_max_min_diff'] = dat_new_fea['cat_max'] - dat_new_fea['cat_min']
    dat_new_fea['cat_max_min_ratio'] = dat_new_fea['cat_min'] / dat_new_fea['cat_max']

    dat_new_fea['cat_idxmax'] = train_cat.idxmax(axis=1)
    dat_new_fea['cat_idxmax'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmax'] = encoder.transform(dat_new_fea['cat_idxmax'])
    dat_new_fea['cat_idxmin'] = train_cat.idxmin(axis=1)
    dat_new_fea['cat_idxmin'].fillna('NaN', inplace=True)
    dat_new_fea['cat_idxmin'] = encoder.transform(dat_new_fea['cat_idxmin'])
    return dat_new_fea



def encode_categorical_by_dep_var(train, test, dep_var_column='Response'):
    for col_name in train.columns:
        if col_name == dep_var_column:
            continue
        dep_var_mean = train[[col_name, dep_var_column]].groupby(col_name).mean()
    
        dep_var_dict = {}
        for level in dep_var_mean.index.tolist():
            dep_var_dict[level] = dep_var_mean.ix[level, dep_var_column]
    
        train[col_name] = train[col_name].replace(dep_var_dict)  
        test[col_name] = test[col_name].replace(dep_var_dict)  

In [8]:
start_time = time.time()

train_cat['Response'] = train_num['Response']
encode_categorical_by_dep_var(train_cat, test_cat)
train_cat.drop('Response', axis=1, inplace=True)

train_cat_Basics = BasicCat_FeatureEngineering(train_cat)
test_cat_Basics  = BasicCat_FeatureEngineering(train_cat)

print 'finish generating categorical features using {} seconds'.format(round(time.time() - start_time, 0))


finish generating categorical features using 68.0 seconds


In [9]:
print train_cat.shape, test_cat.shape
train_cat.head()

(150000, 1294) (150000, 1294)


,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
Id,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
start_time = time.time()
combined_train_cat = pd.concat([train_cat, train_cat_Basics], axis=1)
combined_test_cat  = pd.concat([test_cat, test_cat_Basics], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} seconds'.format(round((time.time() - start_time), 2))

finish feature engineering date using 3.3 seconds


In [11]:
#### numerical feature engineering work
train_num_Basics = NumericalFeatureEngineering(train_num)
test_num_Basics = NumericalFeatureEngineering(test_num)

combined_train_num = pd.concat([train_num, train_num_Basics], axis=1)
combined_test_num  = pd.concat([test_num, test_num_Basics], axis=1)                                                                            
print 'combined train numerical feature shape: {}, combined test numerical features shape: {}'.format(combined_train_num.shape, combined_test_num.shape)

combined train numerical feature shape: (150000, 978), combined test numerical features shape: (150000, 977)


In [12]:
### section of date features

## basic features from tmp_train_dat
train_dat_Basics = BasicDate_FeatureEngineering(train_dat)
test_dat_Basics  = BasicDate_FeatureEngineering(test_dat)

In [13]:
## normalized date columns
train_dat_Norm = train_dat.apply(getRelativeTimeColumns, axis=1)
test_dat_Norm  = test_dat.apply(getRelativeTimeColumns, axis=1)
## remove single-valued columns
remove_single_value_columns(train_dat_Norm, test=test_dat_Norm)

raw train data dimension:  (150000, 1150)
raw test data dimension:  (150000, 1150)
processed train data dimension:  (150000, 1067)
processed test data dimension:  (150000, 1067)


In [14]:
encoder = preprocessing.LabelEncoder()
column_names = train_dat.columns.tolist()
column_names.append('NaN')
encoder.fit(column_names)

LabelEncoder()

In [15]:
## TimeDiff features
train_dat_TimeDiff = train_dat.apply(getTimeChangeColumns, axis=1)
test_dat_TimeDiff  = test_dat.apply(getTimeChangeColumns, axis=1)
TimeDiff_ColumnNames = ['time_diff_start_col', 'time_diff_end_col', 'time_diff_value',
                        'time_ratio_value', 'first_time_value', 'last_time_value', 'first_date_value']
train_dat_TimeDiff.columns = TimeDiff_ColumnNames
test_dat_TimeDiff.columns = TimeDiff_ColumnNames

for column in ['time_diff_start_col', 'time_diff_end_col']:
    train_dat_TimeDiff[column].fillna('NaN', inplace=True)
    train_dat_TimeDiff[column] = encoder.transform(train_dat_TimeDiff[column])
    
    test_dat_TimeDiff[column].fillna('NaN', inplace=True)
    test_dat_TimeDiff[column] = encoder.transform(test_dat_TimeDiff[column])


In [16]:
start_time = time.time()

## section to create timeStep features

unique_value_counts = 6
timeStep_columnNames = []
column_name_columns = []
for i in xrange(unique_value_counts):
    timeStep_columnNames.extend(['time_diff_step_{}'.format(i), 'column_counts_step_{}'.format(i),
                                 'time_cost_step_{}'.format(i), 'first_column_step_{}'.format(i)])
    column_name_columns.append('first_column_step_{}'.format(i))

train_dat_TimeStep = train_dat_Norm.apply(getTimeSteps, axis=1)
test_dat_TimeStep  = test_dat_Norm.apply(getTimeSteps, axis=1)
train_dat_TimeStep.columns = timeStep_columnNames
test_dat_TimeStep.columns  = timeStep_columnNames

for column in column_name_columns:
    train_dat_TimeStep[column].fillna('NaN', inplace=True)
    test_dat_TimeStep[column].fillna('NaN', inplace=True)
    train_dat_TimeStep[column] = encoder.transform(train_dat_TimeStep[column])
    test_dat_TimeStep[column] = encoder.transform(test_dat_TimeStep[column])


print 'finish generating TimeStep features using {} seconds'.format(round(time.time() - start_time, 0))


finish generating TimeStep features using 860.0 seconds


In [17]:
start_time = time.time()
combined_train_dat = pd.concat([train_dat_Norm, train_dat_Basics, train_dat_TimeDiff, train_dat_TimeStep], axis=1)
combined_test_dat  = pd.concat([test_dat_Norm, test_dat_Basics, test_dat_TimeDiff, test_dat_TimeStep], axis=1)                                                                                                                                                 
print 'finish feature engineering date using {} minutes'.format(round((time.time() - start_time)/60, 2))

finish feature engineering date using 0.03 minutes


In [18]:
print combined_train_dat.shape, combined_test_dat.shape
combined_train_dat.head()

(150000, 1108) (150000, 1108)


,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,L0_S0_D19,...,time_cost_step_3,first_column_step_3,time_diff_step_4,column_counts_step_4,time_cost_step_4,first_column_step_4,time_diff_step_5,column_counts_step_5,time_cost_step_5,first_column_step_5
Id,,,,,,,,,,,,,,,,,,,,,
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.257500,1013,5.04,15,0.336000,1018,5.05,14,0.360714,1033
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,1150,NaN,0,0.000000,1150,NaN,0,0.000000,1150
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.083971,945,5.72,29,0.197241,1018,NaN,0,0.000000,1150
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.072500,945,4.94,10,0.494000,1018,4.95,5,0.990000,1028
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.224667,1018,3.38,14,0.241429,1041,NaN,0,0.000000,1150


In [19]:
train_test_datIndex_features = build_IndexFeatures(combined_train_dat, combined_test_dat)

#### combine all the features together

In [20]:
#combined_train = pd.concat([train_dat_stations, train_num_stations, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([test_dat_stations, test_num_stations, combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [44]:
#combined_train = pd.concat([combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [45]:
## combined data with categorical features
#combined_train = pd.concat([combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
#combined_test  = pd.concat([combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [79]:
## combined data with categorical features
combined_train = pd.concat([train_dat_stations, combined_train_cat, combined_train_num, combined_train_dat, train_test_datIndex_features.ix[combined_train_num.index, :]], axis=1)
combined_test  = pd.concat([test_dat_stations, combined_test_cat,  combined_test_num,  combined_test_dat,  train_test_datIndex_features.ix[combined_test_num.index, :]], axis=1)

In [46]:
print combined_test.shape
combined_test.head()

(300000, 3411)


,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,165388.000000,154442.000000,-178143.0,-3232.0,-297775.0,-3232.0,-138624.0,-3232.0,-64614.0,-1.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,79723.500000,93350.500000,-208741.0,-62612.0,-274602.0,-5943.0,-274602.0,-61348.0,1.0,-1.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13435.333333,68460.666667,-283350.0,-24124.0,-290694.0,-17769.0,-255142.0,-20967.0,1.0,-3.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8061.000000,6311.800000,-254439.0,-140294.0,-296594.0,-130413.0,-299030.0,-4697.0,-296594.0,-33948.0


In [47]:
print combined_train.shape
combined_train.head()

(150000, 3412)


,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,L0_S2_F43,...,first_date_value_index_ratio_1,first_date_value_index_ratio_2,time_ratio_value_index_diff_0,time_ratio_value_index_diff_1,first_time_value_index_diff_0,first_time_value_index_diff_1,last_time_value_index_diff_0,last_time_value_index_diff_1,first_date_value_index_diff_0,first_date_value_index_diff_1
Id,,,,,,,,,,,,,,,,,,,,,
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,66236.500000,70779.000000,-249490,-232358,-270686,-47255,-290018,-3482,-298351,-224451
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45512.000000,46202.666667,-174409,-79568,-299900,-67530,-244431,-58861,3,-2
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39010.428571,40557.857143,-284206,-38624,-242935,-26876,-211070,-13283,-242935,-26876
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13113.000000,13969.333333,-299775,-3564,-297979,-3564,-297979,-3564,-297979,-3564
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10598.181818,12043.727273,-61721,-13963,-111468,-1,-62044,-13963,-111468,-1


#### KFold cross-validation

In [ ]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False


In [78]:

skf = StratifiedKFold(combined_train[dep_var_name], 4)

for train_index, valid_index in skf:
    valid_data = combined_train.iloc[valid_index]
    tmp_train  = combined_train.iloc[train_index]

    y = tmp_train[dep_var_name].values
    X = tmp_train.drop(dep_var_name, axis=1)

    valid_y = valid_data[dep_var_name].values
    valid_X = valid_data.drop(dep_var_name, axis=1)
    
    model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='test_bosch_xgb_model')
    model.fit(tmp_train, dep_var_name)
    
    pred = model.predict(valid_X)
    print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
    print '\n'
    print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 178.854316547
a base_score 0.00556005560056 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.896441
[50]	train-auc:0.937568
[100]	train-auc:0.943275
[150]	train-auc:0.94911
[200]	train-auc:0.955527
[250]	train-auc:0.960549
[300]	train-auc:0.964863
[350]	train-auc:0.968634
[400]	train-auc:0.972311
[450]	train-auc:0.975426
[500]	train-auc:0.978361
the xgboost fit is finished by using 450.512493134 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00557988840223
threshold for preds: 0.0800262535407
0.293549122368


result from using flexsible threshold: (0.39161689425379753, 0.2355901300907135)
scale_pos_weight: 178.533213645
a base_score 0.00557 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.896704
[50]	train-auc:0.933447
[100]	train-auc:0.945668
[150]	train-auc:0.952298
[200]	train-auc:0.958971
[250]	train-auc:0.964867
[300]	train-auc:0.969198
[350]	train-auc:0.973281
[400]	train-auc:0.976447
[450]	train-auc:0.979087
[500]	train-auc:0.981173
the xgboost fit is finished by using 444.810596943 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00556
threshold for preds: 0.101060003942
0.305491048214


result from using flexsible threshold: (0.4295261215878571, 0.3729596436023712)
scale_pos_weight: 178.535008977
a base_score 0.00556994430056 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.856659
[50]	train-auc:0.934926
[100]	train-auc:0.944973
[150]	train-auc:0.954006
[200]	train-auc:0.960216
[250]	train-auc:0.965486
[300]	train-auc:0.970324
[350]	train-auc:0.974069
[400]	train-auc:0.977672
[450]	train-auc:0.980344
[500]	train-auc:0.982367
the xgboost fit is finished by using 453.477813959 seconds, saved into test_bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00556011120222
threshold for preds: 0.0743836304009
0.319959908664


result from using flexsible threshold: (0.46605411329989066, 0.2215878814458847)


In [48]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)

In [49]:
params = {}
params["eta"]                      = 0.0075
params["subsample"]                = 0.8
params["colsample_bytree"]         = 0.8
params["num_round"]                = 501
params["max_depth"]                = 5
params["gamma"]                    = 0
params["metrics"]                  = 'auc'
params['eval_metric']              = 'auc'
params["seed"]                     = 999
params['verbose_eval']             = 50
## whether to use weights
params['use_base_score']           = True
params['use_weights']              = True
#params['use_scale_pos_weight']     = True
params["val"]                      = False

model = xgboost_classifier(label_name = dep_var_name, params = params, model_file='bosch_xgb_model')
model.fit(tmp_train, dep_var_name)

scale_pos_weight: 178.488888889
a base_score 0.00557137551071 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.894678
[50]	train-auc:0.935115
[100]	train-auc:0.942989
[150]	train-auc:0.948968
[200]	train-auc:0.953416
[250]	train-auc:0.959547
[300]	train-auc:0.964845
[350]	train-auc:0.969812
[400]	train-auc:0.973346
[450]	train-auc:0.976445
[500]	train-auc:0.978809
the xgboost fit is finished by using 114.877979994 seconds, saved into bosch_xgb_model


In [50]:
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00555567901509
threshold for preds: 0.0849152385588
0.308156338689


result from using flexsible threshold: (0.4084263987220095, 0.24640917778015137)


In [51]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)

model.fit(tmp_train, dep_var_name)
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 178.488888889
a base_score 0.00557137551071 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.894655
[50]	train-auc:0.93618
[100]	train-auc:0.941446
[150]	train-auc:0.948078
[200]	train-auc:0.952912
[250]	train-auc:0.959198
[300]	train-auc:0.965078
[350]	train-auc:0.969878
[400]	train-auc:0.973686
[450]	train-auc:0.976669
[500]	train-auc:0.978909
the xgboost fit is finished by using 115.617825031 seconds, saved into bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00555567901509
threshold for preds: 0.0845989154543
0.324245726161


result from using flexsible threshold: (0.4084263987220095, 0.269706666469574)


In [52]:
############## Section of regular validation #######################
train_index, valid_index = create_validation_index(combined_train, 0.3, dep_var_name, True)
valid_data = combined_train.ix[valid_index]
tmp_train  = combined_train.ix[train_index]

y = tmp_train[dep_var_name].values
X = tmp_train.drop(dep_var_name, axis=1)

valid_y = valid_data[dep_var_name].values
valid_X = valid_data.drop(dep_var_name, axis=1)

model.fit(tmp_train, dep_var_name)
pred = model.predict(valid_X)

print 'result from using constant fraction: \n', score_MCC(valid_y, pred)
print '\n'
print 'result from using flexsible threshold:', CombinedModel.mcc_eval_func(valid_y, pred)

scale_pos_weight: 178.488888889
a base_score 0.00557137551071 is used in the xgboost model...



####################
 train the xgboost without early stopping
####################


[0]	train-auc:0.884097
[50]	train-auc:0.934309
[100]	train-auc:0.944087
[150]	train-auc:0.949238
[200]	train-auc:0.953798
[250]	train-auc:0.959454
[300]	train-auc:0.965253
[350]	train-auc:0.970138
[400]	train-auc:0.973685
[450]	train-auc:0.976442
[500]	train-auc:0.978809
the xgboost fit is finished by using 232.534298897 seconds, saved into bosch_xgb_model
in the prediction step, dep_var_name is not provided....
result from using constant fraction: 
mean of groud truth: 0.00555567901509
threshold for preds: 0.0861773046355
0.300111644953


result from using flexsible threshold: (0.4084263987220095, 0.27156344056129456)
